Delayed computation
===================
Dask allows to split a workload (python function calls) to multiple workers using `Futures`.
However they are hard to use with tasks that are not totally independant. 

`dask.delayed` allows to submit tasks similarly to futures, but in a more pythonic way:   
Decorating function with `delayed`, they then can be used normally.     
 It's output can be used for other delayed input before being computed.  

`delayed` can distribute many normal python workflow if written in functional style.


In [ ]:
from dask import delayed, compute
from dask.distributed import Client
import numpy as np

client = Client(processes=True)
client

## Basic usage

In [ ]:
def func(a, b, c):
    return (2 * a) + (b - c)


a, b, c = 3, 4, 5
func(a, b, c)

In [ ]:
@delayed
def lazy_func(a, b, c):
    return (2 * a) + (b - c)

func_delayed = lazy_func(a, b, c)
func_delayed

The output is a `Delayed` object.  Computation are not started yet.  
Only when `compute` is called, it is passed to the workers.

In [ ]:
func_delayed.compute()

Outputs can be used in other delayed functions without being computed.

In [ ]:
from operator import add, sub, mul


e = delayed(mul)(2, a)
f = delayed(sub)(b, c)
g = delayed(add)(e, f)
g

Operator on delayed also work as expected:

In [ ]:
h = e + f
h

When an `delayed` is build as such from multiple task, `dask` will remember all operations in a tree which can be shown with `visualize`:

In [ ]:
g.visualize()

In [ ]:
g.compute(), h.compute()

Delayed Data
------------
When some data is reused as input in multiple delayed function, it can be set as delayed to help the scheduler.
Delayed data class method can be used.

In [ ]:
# Load the string in the operation tree
var1 = delayed("12345")
var1

In [ ]:
var2 = delayed( lambda N: [1]*N ) (5)
var2

In [ ]:
#Operation between function output and delayed data
oper1 = var1[2] + var2[3] + var1[0]
oper1

In [ ]:
oper1.visualize(rankdir="LR")

In [ ]:
oper1.compute()

## Side effect
**Side effect should not be used!**

In [ ]:
l = []
tasks = [delayed(l.append)(i) for i in range(10)]
compute(tasks) # Better  [task.compute() for task in tasks]
l

In [ ]:
l = delayed([])
tasks = [l.append(i) for i in range(10)]
compute(tasks)
l.compute()

## Data creation

With large data, we need to be careful that the data is not all in the client's memory.  
Large dataset should be created / loaded in a delayed task.

In [ ]:
@delayed
def make_sample(N):
    return np.expm1(5 * np.random.rand(N)**3)

samples = [make_sample(1000) for _ in range(5)]

stds = [delayed(np.std)(sample) for sample in samples]
std_avg = delayed(np.mean)(stds)
std_avg.visualize(rankdir="LR")

In [ ]:
std_avg.compute()

## Exercise: Putting it all together

Use `dask.delayed` to compute the following workflow:

- Open an image.
- Take `N` random 100x100 pixels blocks.
- Compute the differences between each pairs of blocks.
- Make an histogram of these differences.

In [ ]:
from PIL import Image
import itertools

im = np.array(Image.open("images/exemple.png"))[:, :, :3]


def crop(im: np.ndarray, x: int, y: int):
    """
    Cut a 100x100 block in the image starting at x, y (as fraction).
    """
    Nx, Ny, _ = im.shape
    x0 = int(x * (Nx-100))
    y0 = int(y * (Ny-100))
    return im[x0:x0+100, y0:y0+100]


def diff(im1: np.ndarray, im2: np.ndarray):
    """
    Compute the difference between images
    """
    return np.abs(im1 - im2)


def build_histogram(im, N):
    """
    Create an histogram of the difference between
    pairs of N random block in the image.
    """
    xs = np.random.random(N)
    ys = np.random.random(N)
    
    pieces = [
        crop(im, x, y)
        for x, y in zip(xs, ys)
    ]
    
    diffs = [
        diff(im1, im2)
        for im1, im2 in itertools.combinations(pieces, 2)
    ]
    
    means = [np.mean(diff) for diff in diffs]
    
    return np.histogram(means)

build_histogram(im, 50)

## Solution

<!---
im = delayed(Image.open)("images/exemple.png")
im = delayed(lambda I: np.array(I)[:, :, :3])(im)

@delayed
def crop(im: np.ndarray, x: int, y: int):
    """
    Cut a 100x100 block in the image starting at x, y (as fraction).
    """
    Nx, Ny, _ = im.shape
    x0 = int(x * (Nx-100))
    y0 = int(y * (Ny-100))
    return im[x0:x0+100, y0:y0+100]

@delayed
def diff(im1: np.ndarray, im2: np.ndarray):
    """
    Compute the difference between images
    """
    return np.abs(im1 - im2)

# Do not delay a function that call other function delayed
def build_histogram(im, N):
    """
    Create an histogram of the difference between
    pairs of N random block in the image.
    """
    xs = np.random.random(N)
    ys = np.random.random(N)
    
    pieces = [
        crop(im, x, y)
        for x, y in zip(xs, ys)
    ]
    
    diffs = [
        diff(im1, im2)
        for im1, im2 in itertools.combinations(pieces, 2)
    ]
    
    means = [delayed(np.mean)(diff) for diff in diffs]
    
    histogram = delayed(np.histogram)(means)
    return histogram

histogram = build_histogram(im, 4)
histogram.visualize(rankdir="LR")

histogram = build_histogram(im, 50)
%time histogram.compute()
--->

Exercise: Block randomize an image
==================================

1. Open an image.
2. Split it in blocks.
3. Applyt a filter each blocks.
4. Randomize the positions of each blocks.
5. Save it to a file.


In [ ]:
from PIL import ImageFilter
from glob import glob

In [ ]:
files = glob("images/*.png")
block_size = 200

for file in files:
    # Image.open does not load the full image at once, but only when needed.
    image = Image.open(file)
    
    # Compute number of block and block size
    num_blocks = (image.size[0] // block_size, image.size[1] // block_size)
    out_size = (num_blocks[0] * block_size, num_blocks[1] * block_size)
    
    # Cut borders so the image is a multiple of block_size pixels
    left_border = (image.size[0] - out_size[0]) // 2
    right_border = out_size[0] + left_border
    up_border = (image.size[1] - out_size[1]) // 2
    down_border = out_size[1] + up_border
    image = image.crop((left_border, up_border, right_border, down_border))
    
    # Create the output image
    new = Image.new("RGBA", out_size)

    # Create output shuffle order
    shuffled_order = np.arange(num_blocks[0] * num_blocks[1])
    np.random.shuffle(shuffled_order)

    # Paste each block in the new image
    for old_idx, new_idx in enumerate(shuffled_order):
        x, y = np.unravel_index(old_idx, num_blocks)
        # Get the block
        block = image.crop(
            (x * block_size, y * block_size, (x+1)*block_size, (y+1)*block_size)
        )
        # Filter it
        block = block.filter(ImageFilter.SHARPEN)
        x, y = np.unravel_index(new_idx, num_blocks)
        # Insert it
        new.paste(block, (x * block_size, y * block_size))
    # Save the file
    new.save(f"schuffled_{file.split('.')[0][7:]}.png")

Solution
--------
<!-- 
files = glob("images/*.png")
block_size = 200
tasks = []

for file in files:
    image = delayed(Image.open)(file)
    size = Image.open(file).size
    
    # Compute number of block and block size
    num_blocks = (size[0] // block_size, size[1] // block_size)
    out_size = (num_blocks[0] * block_size, num_blocks[1] * block_size)
    
    # Cut borders so the image is a multiple of block_size pixels
    left_border = (image.size[0] - out_size[0]) // 2
    right_border = out_size[0] + left_border
    up_border = (image.size[1] - out_size[1]) // 2
    down_border = out_size[1] + up_border
    image = image.crop((left_border, up_border, right_border, down_border))
    image.visualize()
    
    # Create the output image
    new = delayed(Image.new)("RGBA", out_size)

    # Create output shuffle order
    shuffled_order = np.arange(num_blocks[0] * num_blocks[1])
    np.random.shuffle(shuffled_order)

    @delayed
    def insert(image, block, x, y, block_size):
        image.paste(block, (x * block_size, y * block_size))
        return image
    
    # Paste each block in the new image
    for old_idx, new_idx in enumerate(shuffled_order):
        x, y = np.unravel_index(old_idx, num_blocks)
        # Get the block
        block = delayed(image.crop)(
            (x * block_size, y * block_size, (x+1)*block_size, (y+1)*block_size)
        )
        # Filter it
        block = block.filter(ImageFilter.SHARPEN)
        x, y = np.unravel_index(new_idx, num_blocks)
        # Insert it
        new = insert(new, block, x, y, block_size)

    # Save the file
    tasks.append(delayed(new.save)(f"schuffled_{file.split('.')[0].split('/')[1]}.png"))

compute(tasks)
-->